<h3>General Imports</h3>

In [ ]:
import os
import time

# Mantid imports
from mantid.simpleapi import LoadNexusProcessed, mtd

# drtsans imports
from drtsans.pixel_calibration import calculate_barscan_calibration, load_calibration, as_intensities
from drtsans.mono.biosans.simulated_intensities import clone_component_intensities

In [ ]:
# "plot_wing_detector" is a utility plotting function that we will use a couple of times
%matplotlib inline
#%matplotlib widget
from drtsans.plots import plot_detector
def plot_midrange_detector(input_workspace, axes_mode='tube-pixel', panel_name='midrange_detector'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode,
                         panel_name=panel_name, imshow_kwargs={})

<h3>Dataset</h3>

The barscan are a group of files for BIOSANS midrange detector.

We assume formula $y = dcal - 640$ to translate the position of the bar ($dcal$) stored in the metadata of the runs, and the position of the bar in the frame of reference of the sample ($y$).

In [ ]:
data_dir = '/SNS/EQSANS/shared/sans-backend/data/ornl/sans/hfir/biosans/pixel_calibration/scans/'

first_run, last_run = 4895, 5009

detector_array = 'midrange_detector'  # calibration for the midrange detector
formula = '{y} - 565'  # translate from scan log value to Y-coordinate in the sample's reference frame.
# Gather all file paths into a python list
barscan_files = [os.path.join(data_dir, f'CG3_{run}.nxs') for run in range(first_run, 1 + last_run)]

<h3>Simulated barscan workspaces</h3>

We create the `barscan_workspaces` for the midrange detector by cloning the intensities of the barscan files for the wing detector. Addtionally, we find the middle scan workspace from all barscan workspaces. 

In [ ]:
barscan_workspaces = []
wing_barscan_workspace = mtd.unique_hidden_name()
middle_scan_index = int((last_run - first_run) / 2)
middle_scan_w = None
for scan_index, scan_data in enumerate(barscan_files):
    LoadNexusProcessed(scan_data, OutputWorkspace=wing_barscan_workspace)
    barscan_workspace = clone_component_intensities(
        wing_barscan_workspace,
        output_workspace=mtd.unique_hidden_name(),
        input_component="wing_detector",
        output_component="midrange_detector",
        copy_logs=True,
    )
    if (middle_scan_index == scan_index):
        middle_scan_w = barscan_workspace
    barscan_workspaces.append(str(barscan_workspace))

<h3>Caveats</h3>
As it turns out, the bar is not covering the first and last tubes. We verify this by plotting the intensities on the midrange detector for the middle scan.

We mask the first and last tubes using the correspoding detector ids by looking into the instrument structure. This flags these tubes as faulty when performing the barscan calculations. **Average** pixel positions and heights will be used to estimate the calibration of these faulty tubes.

In [ ]:
plot_midrange_detector(middle_scan_w)

tube_pixels = 256
# mask the last tube in the midrange detector (back-midrange panel/bank104/tube4 from visual inspection of instrument)
closest_masked_detector_ids = list(range(106240, 106240+tube_pixels+1))

#bank 89/tube 1, the furthest from the beam
furthest_masked_detector_ids = list(range(90112, 90112+tube_pixels+1))
masked_detectors = closest_masked_detector_ids + furthest_masked_detector_ids

<h3>Calibration</h3>
We carry out the calibration with the <code>barscan_workspaces</code> in the following cell.

Calculation of the calibration is slow, about 5 to 10 minutes. However, calibrations are carried out once or twice in a cycle.

In [ ]:
# Carry out the calibration

start_time = time.time()
calibration,_ = calculate_barscan_calibration(barscan_workspaces, component=detector_array,
                                            formula=formula, mask=masked_detectors, inspect_data=True)
print('Calibration took ', int((time.time() - start_time) / 60), 'minutes')

<h3>Saving the Calibration</h3>
There are default database files for each instrument when saving a calibration. For BIOSANS is <code>/HFIR/CG3/shared/calibration/pixel_calibration.json</code>. We don't want to mess with the official database so we save this calibration to a temporary database file locally in the <code>/tmp/</code> directory.
We use argument overwrite=True in case we run the notebook more than once. Then we will overwrite the existing 
calibration entry in the database

In [ ]:
#calibration.save(database='/HFIR/CG3/shared/tmp/calibration.json', overwrite=True)
database_path = '/tmp/calibration_midrange_detector.json'
calibration.save(database=database_path, overwrite=True)

<h3>Applying a Saved Calibration</h3>
Next we use one of the barscan workspaces as the target for our recently saved calibration.
Notice that we are applying the calibration to our input workspace and saving the result to an output workspace. If you want to <b>overwrite</b> the input workspace, then omit the <code>output_workspace</code> argument.

In [ ]:
random_workspace = middle_scan_w
saved_calibration = load_calibration(random_workspace, 'BARSCAN', component='midrange_detector',
                                     database=database_path)
start_time = time.time()
saved_calibration.apply(random_workspace, output_workspace='output_workspace')
print('Applying the calibration took ', time.time() - start_time, 'seconds')

<h3>Visualizing the Effects of the Calibration</h3>
We plot intensities on the midrange detector before and after the calibration. Notice we use argument <code>axes_mode='xy'</code> that instruct <code>plot_midrange_detector</code> to plot intensities versus $X$ and $Y$ coordinates, instead of the default plotting. The default plotting is versus tube index and pixel index.

Notice: generating the plots versus $X$ and $Y$ will take about a minute

In [ ]:
plot_midrange_detector(random_workspace, axes_mode='xy')  # before calibration
plot_midrange_detector('output_workspace', axes_mode='xy')  # after calibration

If the calibration has worked, the bar should appear more **levelled** after the calibration.

Also notice that all values of $X$ are negative, as the wing detector is standing on the negative side of the X-axis

<h3>Viewing the Calibrated Positions and Heights</h3>
We can retrieve the pixel positions and heights from the calibrated <code>output_workspace</code>. The cell below will take for each pixel its vertical position and store this value as the intensity value in workspace <code>views.positions</code>. Later we can color plot this workspace to view the assigned positions to each pixel. The same process is done for pixel heights.

Again, extracting the ~50K positions and heights takes time.

In [ ]:
views = as_intensities('output_workspace', component='midrange_detector')
plot_midrange_detector(views.positions)
plot_midrange_detector(views.heights)